# Config variables

In [39]:
############# CHANGE HERE FOR YOUR PURPOSE ####################
# mode = "private data"
mode = "public data" # Select from "private data" or "public data"
data_dir = f"G:\Shared drives\Perceptual model review\ForRyoko\data"
####################################################################

if mode =="private data":
    out_file_name = "giantTable_private"
elif mode =="public data":
    out_file_name = "giantTable_public"
print(f"Start creating {mode}. \nThe output file name will be '{out_file_name}.csv'")

Start creating public data. 
The output file name will be 'giantTable_public.csv'


# Build database (Include all models: Figure, Text, and Hand-drawn)
This scripts can be "Run All" if you don't need to see step-by-step results

In [40]:
import os
import numpy as np
import pandas as pd
import configparser
from sqlalchemy import create_engine
from sqlalchemy import text

## Read datasheets

In [41]:
df_loc = pd.read_excel(os.path.join(data_dir, 'Location_formatted.xlsx')) # The lat/lon should be pre-formatted in decimal units
# df_model = pd.read_csv('../data/ModelAnalysis.csv', encoding='utf-8')
df_model_fig = pd.read_excel(os.path.join(data_dir, 'ModelAnalysis_Figure.xlsx'))
df_model_text = pd.read_excel(os.path.join(data_dir, 'ModelAnalysis_Text.xlsx'))
df_taxonomy = pd.read_excel(os.path.join(data_dir, 'ProcessHierarchyNetwork.xlsx'))
df_FunctionType = pd.read_excel(os.path.join(data_dir, 'FunctionType.xlsx'))
df_model_type = pd.read_excel(os.path.join(data_dir, 'ModelType.xlsx'))
print(f"Total models: {len(df_model_text)+len(df_model_fig)}")
print(f"Text models: {len(df_model_text)}")
print(f"Figure models: {len(df_model_fig)}")


Total models: 396
Text models: 266
Figure models: 130


## Organize data sheets

## Format misc tables into dataframe

In [42]:
# Location table
# Sanity check if data can be joined
# df.set_index('key').join(other.set_index('key'))
df_loc["id"] = df_loc.index + 1
df_loc = df_loc.drop(columns='Unnamed: 0')
df_loc["huc_watershed_id"] = np.nan

# FunctionType table
df_FunctionType["id"] = df_FunctionType.index + 1

# Model type tale
df_model_type["id"] = df_model_type.index + 1

# Alternative name table
df_altNames0 = df_taxonomy.set_index(['process', 'function', 'identifier', 'process_level']).apply(
    lambda x: x.str.split(',').explode()).reset_index()
df_altNames = df_altNames0[['alternative_names', 'process']].copy()
df_altNames['alternative_names'] = df_altNames['alternative_names'].str.strip()
df_altNames['alternative_names'] = df_altNames['alternative_names'].str.capitalize()
df_altNames.dropna(axis=0, inplace=True)
df_altNames["id"] = df_altNames.index + 1

### Fill nan data in "figure_url" with corgi picture

In [43]:
# corgi_fig = "https://i.ibb.co/stdkk6P/corgi-with-notes.png"
corgi_fig = "https://sdsugeo.maps.arcgis.com/sharing/rest/content/items/df499e37a2ef40d7885966952c627e01/data"
df_model_fig['figure_url'] = df_model_fig['figure_url'].fillna(corgi_fig) # No attribution URL 

### Public vs. private version

In [44]:
if mode == "public data":
    df_model_fig['figure_url'][df_model_fig['attribution']=="Not open-access"] = corgi_fig  # There is attribution but Not-open access
    df_model_text['textmodel_snipped'][df_model_text['attribution']=="Not open-access"] = "Not open-access. See article for the Text Model."

C:\Users\flipl\AppData\Local\Temp\ipykernel_11560\4084942147.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model_fig['figure_url'][df_model_fig['attribution']=="Not open-access"] = corgi_fig  # There is attribution but Not-open access
C:\Users\flipl\AppData\Local\Temp\ipykernel_11560\4084942147.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model_text['textmodel_snipped'][df_model_text['attribution']=="Not open-access"] = "Not open-access. See article for the Text Model."


## Combine all dataframes

In [45]:
# Combine all model dataframes into one "df_model"
df_model_fig['model_type'] = 'Figure model'
df_model_text['model_type']  = 'Text model'
df_model = pd.concat([df_model_fig, df_model_text], join='outer', ignore_index=True)
df_model["id"] = df_model.index + 1

# Select columns that will be the main table in the database
df_modelmain = df_model[['id', 'citation', 'model_type', 'watershed_name',
                        'figure_num', 'figure_url', 'figure_caption',
                        'textmodel_snipped', 'textmodel_section_number', 'textmodel_page_number', 'textmodel_section_name', 
                        'spatial_property', 'num_spatial_zones', 'temporal_property', 
                        'num_temporal_zones', 'vegetation_info', 'soil_info', 'geol_info',
                        'topo_info', 'three_d_info', 'uncertainty_info', 'other_info'
                        ]].copy()

print(f'{len(df_modelmain)-1}')

395


In [46]:
df_modelmain['spatial_property'].fillna('N', inplace=True)
df_modelmain['temporal_property'].fillna('N', inplace=True)
df_modelmain['figure_url'].fillna('N/A', inplace=True)
df_modelmain['figure_caption'].fillna('N/A', inplace=True)
df_modelmain['textmodel_section_number'].fillna('N/A', inplace=True)
df_modelmain['textmodel_page_number'].fillna('N/A', inplace=True)
df_modelmain['textmodel_section_name'].fillna('N/A', inplace=True)
df_modelmain['vegetation_info'].fillna('N', inplace=True)
df_modelmain['soil_info'].fillna('N', inplace=True)
df_modelmain['geol_info'].fillna('N', inplace=True)
df_modelmain['topo_info'].fillna('N', inplace=True)
df_modelmain['three_d_info'].fillna('N', inplace=True)
df_modelmain['uncertainty_info'].fillna('N', inplace=True)
df_modelmain['other_info'].fillna('N', inplace=True)

### Clean attribution

In [47]:
# Mapping between 'attribution' values and URLs
attributions_map = {
    "CC-BY-4.0": "https://creativecommons.org/licenses/by/4.0/",
    "CC-BY 4.0": "https://creativecommons.org/licenses/by/4.0/",
    "CC BY 4.0": "https://creativecommons.org/licenses/by/4.0/",
    "CC-BY": "https://creativecommons.org/licenses/by/4.0/",
    "CC-BY-3.0": "https://creativecommons.org/licenses/by/3.0/",
    "CC-BY 3.0": "https://creativecommons.org/licenses/by/3.0/",
    "CC BY 3.0": "https://creativecommons.org/licenses/by/3.0/",
    "CC-BY-NC": "https://creativecommons.org/licenses/by-nc/4.0/",
    "CC BY-NC": "https://creativecommons.org/licenses/by-nc/4.0/",
    "CC BY NC": "https://creativecommons.org/licenses/by-nc/4.0/",
    "CC-BY-NC-4.0": "https://creativecommons.org/licenses/by-nc/4.0/",
    "CC BY-NC-4.0": "https://creativecommons.org/licenses/by-nc/4.0/",
    "CC BY-NC 4.0": "https://creativecommons.org/licenses/by-nc/4.0/",
    "CC-BY-NC 4.0": "https://creativecommons.org/licenses/by-nc/4.0/",
    "CC BY NC 4.0": "https://creativecommons.org/licenses/by-nc/4.0/",
    "CC-BY-NC-SA": "https://creativecommons.org/licenses/by-nc-sa/2.5/",
    "CC BY-NC-SA": "https://creativecommons.org/licenses/by-nc-sa/2.5/",
    "CC BY-NC-SA 2.5": "https://creativecommons.org/licenses/by-nc-sa/2.5/",
    "CC-BY-NC-ND": "https://creativecommons.org/licenses/by-nc-nd/4.0/",
    "CC BY-NC-ND": "https://creativecommons.org/licenses/by-nc-nd/4.0/",
    "CC BY NC ND": "https://creativecommons.org/licenses/by-nc-nd/4.0/"
}

# Apply the mapping to update 'attribution_url' column
for attribution, url in attributions_map.items():
    df_model.loc[df_model['attribution'] == attribution, "attribution_url"] = url

## Citation table

In [48]:

df_model["attribution_url"].fillna(df_model["url"], inplace=True) # Fill the NaN in the attribution URL with paper URL (i.e., "See paper for Not-open access ones")
df_citation = df_model[["citation", "url"]].copy()
df_citation["attribution"] = df_model["attribution"].copy()
df_citation["attribution_url"] = df_model["attribution_url"].copy()
df_citation["id"] = df_citation.index + 1

## Spatial and temporal zone tables

In [49]:
df_spatialZoneType = df_model["spatial_property"].copy().drop_duplicates()
df_spatialZoneType = df_spatialZoneType.to_frame()
df_spatialZoneType.reset_index(inplace=True)
df_spatialZoneType = df_spatialZoneType.drop(columns='index')
df_spatialZoneType['id'] = df_spatialZoneType.index + 1

df_temporalZoneType = df_model["temporal_property"].copy().drop_duplicates()
df_temporalZoneType = df_temporalZoneType.to_frame()
df_temporalZoneType.reset_index(inplace=True)
df_temporalZoneType = df_temporalZoneType.drop(columns='index')
df_temporalZoneType['id'] = df_temporalZoneType.index + 1

## LinkProcessPerceptual table

In [50]:

# Get all the process original text and taxonomy name from model
frames = [df_model[['id', 'flux1', 'flux1_taxonomy']].copy().rename(
    columns={"id": "entry_id", "flux1": "original_text", "flux1_taxonomy": "process"}),
        df_model[['id', 'flux2', 'flux2_taxonomy']].copy().rename(columns={"id": "entry_id", "flux2": "original_text", "flux2_taxonomy": "process"}),
        df_model[['id', 'flux3', 'flux3_taxonomy']].copy().rename(columns={"id": "entry_id", "flux3": "original_text", "flux3_taxonomy": "process"}),
        df_model[['id', 'flux4', 'flux4_taxonomy']].copy().rename(columns={"id": "entry_id", "flux4": "original_text", "flux4_taxonomy": "process"}),
        df_model[['id', 'flux5', 'flux5_taxonomy']].copy().rename(columns={"id": "entry_id", "flux5": "original_text", "flux5_taxonomy": "process"}),
        df_model[['id', 'flux6', 'flux6_taxonomy']].copy().rename(columns={"id": "entry_id", "flux6": "original_text", "flux6_taxonomy": "process"}),
        df_model[['id', 'flux7', 'flux7_taxonomy']].copy().rename(columns={"id": "entry_id", "flux7": "original_text", "flux7_taxonomy": "process"}),
        df_model[['id', 'flux8', 'flux8_taxonomy']].copy().rename(columns={"id": "entry_id", "flux8": "original_text", "flux8_taxonomy": "process"}),
        df_model[['id', 'flux9', 'flux9_taxonomy']].copy().rename(columns={"id": "entry_id", "flux9": "original_text", "flux9_taxonomy": "process"}),
        df_model[['id', 'flux10', 'flux10_taxonomy']].copy().rename(columns={"id": "entry_id", "flux10": "original_text", "flux10_taxonomy": "process"}),
        df_model[['id', 'flux11', 'flux11_taxonomy']].copy().rename(columns={"id": "entry_id", "flux11": "original_text", "flux11_taxonomy": "process"}),
        df_model[['id', 'flux12', 'flux12_taxonomy']].copy().rename(columns={"id": "entry_id", "flux12": "original_text", "flux12_taxonomy": "process"}),
        df_model[['id', 'flux13', 'flux13_taxonomy']].copy().rename(columns={"id": "entry_id", "flux13": "original_text", "flux13_taxonomy": "process"}),
        df_model[['id', 'flux14', 'flux14_taxonomy']].copy().rename(columns={"id": "entry_id", "flux14": "original_text", "flux14_taxonomy": "process"}),
        df_model[['id', 'store1', 'store1_taxonomy']].copy().rename(columns={"id": "entry_id", "store1": "original_text", "store1_taxonomy": "process"}),
        df_model[['id', 'store2', 'store2_taxonomy']].copy().rename(columns={"id": "entry_id", "store2": "original_text", "store2_taxonomy": "process"}),
        df_model[['id', 'store3', 'store3_taxonomy']].copy().rename(columns={"id": "entry_id", "store3": "original_text", "store3_taxonomy": "process"}),
        df_model[['id', 'store4', 'store4_taxonomy']].copy().rename( columns={"id": "entry_id", "store4": "original_text", "store4_taxonomy": "process"}),
        df_model[['id', 'store5', 'store5_taxonomy']].copy().rename(columns={"id": "entry_id", "store5": "original_text", "store5_taxonomy": "process"}),
        df_model[['id', 'store6', 'store6_taxonomy']].copy().rename(columns={"id": "entry_id", "store6": "original_text", "store6_taxonomy": "process"}),
        df_model[['id', 'store7', 'store7_taxonomy']].copy().rename(columns={"id": "entry_id", "store7": "original_text", "store7_taxonomy": "process"}),
        df_model[['id', 'store8', 'store8_taxonomy']].copy().rename(columns={"id": "entry_id", "store8": "original_text", "store8_taxonomy": "process"}),
        ]

df_linkProcessPerceptual0 = pd.concat(frames, axis=0, ignore_index=True)
df_linkProcessPerceptual0["id"] = df_linkProcessPerceptual0.index + 1

# Create taxonomy table
df_process0 = df_taxonomy.drop(columns='alternative_names')

# join process taxonomy and model table
df_linkProcessPerceptual0["process_lower"] = df_linkProcessPerceptual0['process'].str.lower()
df_linkProcessPerceptual0["process_lower"] = df_linkProcessPerceptual0['process_lower'].str.strip()
df_process0["process_lower"] = df_process0['process'].str.lower()
df_process0["process_lower"] = df_process0['process_lower'].str.strip()

# find and add some new process from model table to taxonomy table (# Check here if you want to check process miscategorization)
df_linkProcessPerceptual1 = df_linkProcessPerceptual0.merge(df_process0, on='process_lower', how='left')
new_process = df_linkProcessPerceptual1.loc[(df_linkProcessPerceptual1['process_x'].isnull() == False) & (
            df_linkProcessPerceptual1['process_y'].isnull() == True)]
new_process.drop_duplicates(subset='process_lower', inplace=True)

add_new_process = pd.DataFrame(
    {'process': new_process['process_x'], 'process_lower': new_process['process_x'].str.lower(),
     'identifier': ['NewProcess'] * len(new_process['process_x'])})
df_process1 = pd.concat([df_process0, add_new_process])
df_process1["id"] = df_process1.reset_index().index + 1

# re-join process taxonomy and model table with new process
df_linkProcessPerceptual2 = df_linkProcessPerceptual0.merge(df_process1, on='process_lower', how='left')
df_linkProcessPerceptual2.rename(columns={"id_y": "process_id"}, inplace=True)
df_linkProcessPerceptual = df_linkProcessPerceptual2.drop(
    columns={'process_x', 'id_x', 'process_lower', 'process_y', 'function', 'identifier', 'process_level'})
df_linkProcessPerceptual.dropna(subset=['original_text'], axis=0, inplace=True)

In [51]:
df_linkProcessPerceptual["id"] = df_linkProcessPerceptual.reset_index().index + 1
df_linkProcessPerceptual["process_id"] = df_linkProcessPerceptual["process_id"].astype('int') 
# If the above line returned an error, it's likely some entry is not finding matching taxonomy. Check the entry by running below
# df_linkProcessPerceptual[df_linkProcessPerceptual['process_id'].isnull()]
df_process = df_process1.drop(columns='process_lower')

In [52]:
print(f'{len(df_modelmain)-1}')

395


## Connect to SQL database

In [53]:
# Read config
config = configparser.ConfigParser()
config.read('config.ini')

DB_NAME = config['postgresql']['DB_NAME']
HOST = config['postgresql']['HOST']
PORT = config['postgresql']['PORT']
USER_NAME = config['postgresql']['USER_NAME']
PASSWD = config['postgresql']['PASSWD']
schema = 'perceptual_model'

# Connect to database
conn_string = f'postgresql+psycopg2://{USER_NAME}:{PASSWD}@{HOST}:{PORT}/{DB_NAME}'
db = create_engine(conn_string, client_encoding='utf8')
try:
    db_auto = db.execution_options(isolation_level="AUTOCOMMIT")
    # This is added from SQL alchemy v2.0
    # You have to COMMIT to put the results into database, and this options allows it
    # See https://docs.sqlalchemy.org/en/20/core/connections.html
    conn = db_auto.connect()
    print("connection to '%s'@'%s' success!" % (DB_NAME, HOST))
except Exception as e:
    print("connection to '%s'@'%s' failed." % (DB_NAME, HOST))
    print(e)

# Initial commands 
conn.execute(text("set search_path to public, perceptual_model"))
conn.execute(text("SET CLIENT_ENCODING TO 'UTF8';"))

# # View the records
# results = conn.execute(text(f"SELECT * from {schema}.locations"))
# for record in results:
#     print("\n", record)

connection to 'postgres'@'localhost' success!


## Create database

In [54]:
# Drop old tables
tables = ['locations', 'citations', 'spatial_zone_type', 'temporal_zone_type', 'process_alt_names',
            'function_type', 'perceptual_model', 'link_process_perceptual', 'process_taxonomy', 'model_type']
for table in tables:
    try:
        conn.execute(text(f"DROP TABLE {schema}.{table} CASCADE;"))
        print(f'Dropped old tables ({schema}.{table})')
    except Exception as e:
        conn.rollback()
        print(e)


Dropped old tables (perceptual_model.locations)
Dropped old tables (perceptual_model.citations)
Dropped old tables (perceptual_model.spatial_zone_type)
Dropped old tables (perceptual_model.temporal_zone_type)
Dropped old tables (perceptual_model.process_alt_names)
Dropped old tables (perceptual_model.function_type)
Dropped old tables (perceptual_model.perceptual_model)
Dropped old tables (perceptual_model.link_process_perceptual)
Dropped old tables (perceptual_model.process_taxonomy)
Dropped old tables (perceptual_model.model_type)


In [55]:
connarg = {'con': conn, 'schema': 'perceptual_model', 'if_exists': 'replace', 'index': False}

# set table names in lower case https://github.com/pandas-dev/pandas/issues/13206
df_loc.to_sql('locations', **connarg)
df_citation.to_sql('citations', **connarg)
df_spatialZoneType.to_sql('spatial_zone_type', **connarg)
df_temporalZoneType.to_sql('temporal_zone_type', **connarg)
df_altNames.to_sql('process_alt_names', **connarg)
df_FunctionType.to_sql('function_type', **connarg)
df_modelmain.to_sql('perceptual_model', **connarg)
df_linkProcessPerceptual.to_sql('link_process_perceptual', **connarg)
df_process.to_sql('process_taxonomy', **connarg)
df_model_type.to_sql('model_type', **connarg)

3

In [56]:
# Location lat lon --> geometry with PostGIS
query = "ALTER TABLE perceptual_model.locations \
ADD COLUMN pt geometry(point, 4326); \
WITH pt_geom AS ( \
	SELECT id, ST_SetSRID(ST_MakePoint(lon, lat), 4326) AS pt \
	from perceptual_model.locations \
	) \
UPDATE perceptual_model.locations \
SET pt = pt_geom.pt \
FROM pt_geom \
WHERE pt_geom.id = locations.id; \
    "

try:
    conn.execute(text(query))
    print("success")
except Exception as e:
    print(e)

success


In [57]:
# Add primary keys
for table in tables:
    try:
        conn.execute(text(f"ALTER TABLE {schema}.{table} ADD PRIMARY KEY (id);"))
        print(f"successfully added primary keys to table {table}")
    except Exception as e:
        print(e)

successfully added primary keys to table locations
successfully added primary keys to table citations
successfully added primary keys to table spatial_zone_type
successfully added primary keys to table temporal_zone_type
successfully added primary keys to table process_alt_names
successfully added primary keys to table function_type
successfully added primary keys to table perceptual_model
successfully added primary keys to table link_process_perceptual
successfully added primary keys to table process_taxonomy
successfully added primary keys to table model_type


In [58]:
# add foreign keys

# model & location
query = "ALTER TABLE perceptual_model ADD COLUMN location_id int; \
UPDATE perceptual_model \
SET location_id = locations.id \
FROM locations \
WHERE perceptual_model.watershed_name = locations.name; \
ALTER TABLE perceptual_model \
ADD FOREIGN KEY (location_id) REFERENCES locations(id); \
ALTER TABLE perceptual_model DROP COLUMN watershed_name;"

try:
    conn.execute(text(query))
    print("success")
except Exception as e:
    print(e)
    
# model & citation
query = "ALTER TABLE perceptual_model ADD COLUMN citation_id int; \
UPDATE perceptual_model \
SET citation_id = citations.id \
FROM citations \
WHERE perceptual_model.citation = citations.citation; \
ALTER TABLE perceptual_model \
ADD FOREIGN KEY (citation_id) REFERENCES citations(id); \
ALTER TABLE perceptual_model DROP COLUMN citation;"

try:
    conn.execute(text(query))
    print("success")
except Exception as e:
    print(e)
    
# model & spatial zone
query = "ALTER TABLE perceptual_model ADD COLUMN spatialzone_id int; \
UPDATE perceptual_model \
SET spatialzone_id = spatial_zone_type.id \
FROM spatial_zone_type \
WHERE perceptual_model.spatial_property = spatial_zone_type.spatial_property; \
ALTER TABLE perceptual_model \
ADD FOREIGN KEY (spatialzone_id) REFERENCES spatial_zone_type(id); \
ALTER TABLE perceptual_model DROP COLUMN spatial_property;"

try:
    conn.execute(text(query))
    print("success")
except Exception as e:
    print(e)
    

# model & temporal zone
query = "ALTER TABLE perceptual_model ADD COLUMN temporalzone_id int; \
UPDATE perceptual_model \
SET temporalzone_id = temporal_zone_type.id \
FROM temporal_zone_type \
WHERE perceptual_model.temporal_property = temporal_zone_type.temporal_property; \
ALTER TABLE perceptual_model \
ADD FOREIGN KEY (temporalzone_id) REFERENCES temporal_zone_type(id); \
ALTER TABLE perceptual_model DROP COLUMN temporal_property;"

try:
    conn.execute(text(query))
    print("success")
except Exception as e:
    print(e)


# model & linktable
query = "ALTER TABLE link_process_perceptual  \
ADD FOREIGN KEY (entry_id) REFERENCES perceptual_model(id);"

try:
    conn.execute(text(query))
    print("success")
except Exception as e:
    print(e)


# process taxonomy & alternative names
query = "ALTER TABLE process_alt_names ADD COLUMN process_id int; \
UPDATE process_alt_names \
SET process_id = process_taxonomy.id \
FROM process_taxonomy \
WHERE process_alt_names.process = process_taxonomy.process; \
ALTER TABLE process_alt_names \
ADD FOREIGN KEY (process_id) REFERENCES process_taxonomy(id); \
ALTER TABLE process_alt_names DROP COLUMN process;"

try:
    conn.execute(text(query))
    print("success")
except Exception as e:
    print(e)
    

# process taxonomy & function type
query = "ALTER TABLE process_taxonomy ADD COLUMN function_id int; \
UPDATE process_taxonomy \
SET function_id = function_type.id \
FROM function_type \
WHERE process_taxonomy.function = function_type.name; \
ALTER TABLE process_taxonomy \
ADD FOREIGN KEY (function_id) REFERENCES function_type(id); \
ALTER TABLE process_taxonomy DROP COLUMN function;"

try:
    conn.execute(text(query))
    print("success")
except Exception as e:
    print(e)
    


success
success
success
success
success
success
success


In [59]:

# model & model type
query = "ALTER TABLE perceptual_model ADD COLUMN model_type_id int; \
UPDATE perceptual_model \
SET model_type_id = model_type.id \
FROM model_type \
WHERE perceptual_model.model_type = model_type.name; \
ALTER TABLE perceptual_model \
ADD FOREIGN KEY (model_type_id) REFERENCES model_type(id); \
ALTER TABLE perceptual_model DROP COLUMN model_type;"

try:
    conn.execute(text(query))
    print("Link model type & perceptual model -- success")
except Exception as e:
    print(e)

Link model type & perceptual model -- success


In [60]:
# try joining on pandas ...
query = "ALTER TABLE link_process_perceptual \
ADD FOREIGN KEY (process_id) REFERENCES process_taxonomy(id);"
# \
# ALTER TABLE link_process_perceptual DROP COLUMN process_name;"

try:
    conn.execute(text(query))
    print("Link taxonomy & perceptual model -- success")
except Exception as e:
    print(e)



Link taxonomy & perceptual model -- success


## Join tables and dump everything into csv file

In [61]:
try:
    conn.execute(text('DROP TABLE giant_table;'))
    print("success")
except Exception as e:
    print(e)

(psycopg2.errors.UndefinedTable) table "giant_table" does not exist

[SQL: DROP TABLE giant_table;]
(Background on this error at: https://sqlalche.me/e/20/f405)


In [62]:
try:
    result = conn.execute(text('SELECT COUNT(*) FROM perceptual_model;'))
    print(result.fetchone()[0])
except Exception as e:
    print(e)

396


In [63]:
join_desired_tables = "\
CREATE TEMP TABLE giant_table AS (\
SELECT perceptual_model.id, \
		model_type.name AS model_type, \
		citations.citation, \
		citations.url, \
        citations.attribution, \
        citations.attribution_url, \
		perceptual_model.figure_num, \
        perceptual_model.figure_caption, \
        perceptual_model.figure_url, \
		perceptual_model.textmodel_snipped, \
        perceptual_model.textmodel_section_number, \
        perceptual_model.textmodel_section_name, \
		perceptual_model.textmodel_page_number, \
        locations.name AS watershed_name, \
		locations.lat, \
		locations.lon, \
        locations.area_km2, \
		process_taxonomy.process, \
		process_taxonomy.identifier,\
		function_type.name AS function_name,\
		perceptual_model.num_spatial_zones,\
		spatial_zone_type.spatial_property,\
		perceptual_model.num_temporal_zones,\
		temporal_zone_type.temporal_property,\
		perceptual_model.vegetation_info,\
		perceptual_model.soil_info,\
		perceptual_model.geol_info,\
		perceptual_model.topo_info,\
		perceptual_model.three_d_info,\
		perceptual_model.uncertainty_info,\
		perceptual_model.other_info\
	FROM perceptual_model \
	INNER JOIN citations \
		ON citations.id = perceptual_model.citation_id \
	INNER JOIN locations \
		ON locations.id = perceptual_model.location_id \
	INNER JOIN spatial_zone_type \
		ON spatial_zone_type.id = perceptual_model.spatialzone_id \
	INNER JOIN temporal_zone_type \
		ON temporal_zone_type.id = perceptual_model.temporalzone_id \
	INNER JOIN link_process_perceptual \
		ON perceptual_model.id = link_process_perceptual.entry_id \
	INNER JOIN process_taxonomy \
		ON link_process_perceptual.process_id = process_taxonomy.id \
	INNER JOIN function_type \
		ON process_taxonomy.function_id = function_type.id \
	INNER JOIN model_type \
		ON perceptual_model.model_type_id = model_type.id \
ORDER BY perceptual_model.id \
);"

try:
    conn.execute(text(join_desired_tables))
    print("Create base giant table -- success")
except Exception as e:
    print(e)

Create base giant table -- success


In [64]:
try:
    result = conn.execute(text('SELECT COUNT(*) FROM perceptual_model;'))
    print(result.fetchone()[0])
except Exception as e:
    print(e)


396


# Join tables and output results

In [65]:
output_dir = os.path.join(data_dir, "for_arcgis_dashboard")
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [66]:
query = "SELECT * FROM perceptual_model"
try:
    df_results = pd.read_sql(text(query), conn)
    # results = conn.execute(query).fetchall()
    print("Fetch results from giant tables -- success")
    print(df_results.columns)
except Exception as e:
    print(e)
df_results.to_csv(os.path.join(output_dir, 'giant_table_debug.csv'), sep=',', header=True, index=False, encoding='utf-8')

Fetch results from giant tables -- success
Index(['id', 'figure_num', 'figure_url', 'figure_caption', 'textmodel_snipped',
       'textmodel_section_number', 'textmodel_page_number',
       'textmodel_section_name', 'num_spatial_zones', 'num_temporal_zones',
       'vegetation_info', 'soil_info', 'geol_info', 'topo_info',
       'three_d_info', 'uncertainty_info', 'other_info', 'location_id',
       'citation_id', 'spatialzone_id', 'temporalzone_id', 'model_type_id'],
      dtype='object')


In [67]:
join_desired_tables2 = "\
CREATE TEMP TABLE giant_table_base AS ( \
SELECT DISTINCT \
    id,  \
	model_type,\
	citation,  \
	url,  \
	attribution,  \
	attribution_url,  \
	figure_num,  \
	figure_caption,  \
	figure_url,  \
	textmodel_snipped, \
	textmodel_section_number, \
	textmodel_section_name, \
	textmodel_page_number, \
	watershed_name,  \
	lat,  \
	lon,  \
	area_km2,  \
	num_spatial_zones,  \
	spatial_property,  \
	num_temporal_zones,  \
	temporal_property,  \
	vegetation_info,  \
	soil_info,  \
	geol_info,  \
	topo_info,  \
	three_d_info,  \
	uncertainty_info,  \
	other_info \
FROM giant_table \
); \
CREATE TEMP TABLE giant_table_flux AS (  \
SELECT DISTINCT  \
id AS temp_id1,  \
COUNT(process) OVER(PARTITION BY id) AS num_flux,  \
STRING_AGG(process, ', ') OVER(PARTITION BY id) AS flux_list,  \
STRING_AGG(identifier, ', ') OVER(PARTITION BY id) AS flux_id_list  \
FROM   giant_table  \
	WHERE function_name ILIKE 'Filling of store'  \
    OR function_name IS NULL\
	OR function_name ILIKE 'Release from store'  \
	OR function_name ILIKE 'In-catchment flux'  \
	OR function_name ILIKE 'In-store flux'  \
	OR function_name ILIKE 'Release'  \
	); \
CREATE TEMP TABLE giant_table_store AS ( \
SELECT DISTINCT id AS temp_id2,  \
COUNT(process) OVER(PARTITION BY id) AS num_store,  \
STRING_AGG(process, ', ') OVER(PARTITION BY id) AS store_list,  \
STRING_AGG(identifier, ', ') OVER(PARTITION BY id) AS store_id_list  \
FROM giant_table  \
	WHERE function_name ILIKE 'Store'  \
	OR function_name ILIKE 'Store, temporary'  \
	OR function_name ILIKE 'Store characteristics, temporary'  \
	OR function_name ILIKE 'Store characteristics, permanent'  \
	);  \
CREATE TEMP TABLE giant_table_update AS ( \
SELECT * FROM giant_table_base  \
LEFT JOIN giant_table_store  \
ON giant_table_base.id = giant_table_store.temp_id2 \
LEFT JOIN giant_table_flux \
ON giant_table_base.id = giant_table_flux.temp_id1 \
ORDER BY id  \
	); \
ALTER TABLE giant_table_update \
DROP COLUMN temp_id1, \
DROP COLUMN temp_id2,  \
ADD COLUMN dummy_column NUMERIC DEFAULT 1; \
"

try:
    conn.execute(text(join_desired_tables2))
    print("Create giant model table with flux & stores -- success")
except Exception as e:
    print(e)

Create giant model table with flux & stores -- success


In [68]:
try:
    result = conn.execute(text('SELECT COUNT(*) FROM giant_table_base;'))
    print(result.fetchone()[0])
except Exception as e:
    print(e)

396


In [69]:
try:
    result = conn.execute(text('SELECT giant_table.id FROM giant_table LEFT OUTER JOIN giant_table_update ON giant_table.id = giant_table_update.id WHERE giant_table_update.id IS NULL;'))
    print(result.fetchone()[0])
except Exception as e:
    print(e)

'NoneType' object is not subscriptable


In [70]:
query = "SELECT * FROM giant_table_update"
try:
    df_results = pd.read_sql(text(query), conn)
    # results = conn.execute(query).fetchall()
    print("Fetch results from giant tables -- success")
    print(df_results.columns)
except Exception as e:
    print(e)


Fetch results from giant tables -- success
Index(['id', 'model_type', 'citation', 'url', 'attribution', 'attribution_url',
       'figure_num', 'figure_caption', 'figure_url', 'textmodel_snipped',
       'textmodel_section_number', 'textmodel_section_name',
       'textmodel_page_number', 'watershed_name', 'lat', 'lon', 'area_km2',
       'num_spatial_zones', 'spatial_property', 'num_temporal_zones',
       'temporal_property', 'vegetation_info', 'soil_info', 'geol_info',
       'topo_info', 'three_d_info', 'uncertainty_info', 'other_info',
       'num_store', 'store_list', 'store_id_list', 'num_flux', 'flux_list',
       'flux_id_list', 'dummy_column'],
      dtype='object')


In [71]:
try:
    result = conn.execute(text('SELECT COUNT(*) FROM giant_table_update;'))
    print(result.fetchone()[0])
except Exception as e:
    print(e)

396


In [72]:
# Some last housekeeping
df_results['huc_watershed_id'] = 'N/A'
df_results['num_store'].fillna(0, inplace=True)
df_results['num_flux'].fillna(0, inplace=True)
df_results['area_km2'].fillna(-9999, inplace=True)
df_results.fillna('N/A', inplace=True)
df_results.to_csv(os.path.join(output_dir, f'{out_file_name}.csv'), sep=',', header=True, index=False, encoding='utf-8')

In [73]:
print(len(df_results))

396


In [74]:
conn.close()
db.dispose()
print(fr'Exported results to ../data/{out_file_name}.csv:')

import datetime
print(datetime.datetime.now())

Exported results to ../data/giantTable_public.csv:
2024-03-16 18:21:02.473221
